In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns


# Đánh giá dữ liệu

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import (
    f1_score, precision_score, recall_score, accuracy_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as plt
import seaborn as sns
import numpy as np

def evaluate_model(model, X_train, y_train, X_test, y_test):
    def evaluate_split(X, y, split_name):
        y_pred = model.predict(X)
        y_proba = model.predict_proba(X)

        f1 = f1_score(y, y_pred, average=None)
        precision = precision_score(y, y_pred, average=None)
        recall = recall_score(y, y_pred, average=None)
        accuracy = accuracy_score(y, y_pred)

        y_bin = label_binarize(y, classes=[0, 1, 2, 3, 4])
        roc_auc = roc_auc_score(y_bin, y_proba, multi_class="ovr")

        print(f"\n--- {split_name} Metrics ---")
        print("F1 Score (per class):", f1)
        print("Precision (per class):", precision)
        print("Recall (per class):", recall)
        print(f"Accuracy: {accuracy:.4f}")
        print(f"AUC (One-vs-Rest): {roc_auc:.4f}")

        print("\nClassification Report:")
        print(classification_report(y, y_pred, target_names=["E", "D", "C", "B", "A"]))

        conf_matrix = confusion_matrix(y, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
                    xticklabels=["E", "D", "C", "B", "A"],
                    yticklabels=["E", "D", "C", "B", "A"])
        plt.title(f"Confusion Matrix - {split_name}")
        plt.xlabel("Predicted Label")
        plt.ylabel("True Label")
        plt.show()

        # ROC Curve
        plt.figure(figsize=(10, 8))
        for i in range(5):
            fpr, tpr, _ = roc_curve(y_bin[:, i], y_proba[:, i])
            auc_score = roc_auc_score(y_bin[:, i], y_proba[:, i])
            plt.plot(fpr, tpr, label=f"Class {i} (AUC = {auc_score:.2f})")
        plt.plot([0, 1], [0, 1], 'k--')
        plt.title(f"ROC Curve - {split_name}")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.legend()
        plt.show()

        return f1, precision, recall, accuracy, roc_auc

    train_metrics = evaluate_split(X_train, y_train, "Train")
    test_metrics = evaluate_split(X_test, y_test, "Test")

    return {
        "train": train_metrics,
        "test": test_metrics
    }

# Permutation Importance

In [ ]:
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import pandas as pd

def plot_permutation_importance(model, X_train, X_train_scaled, y_train):
    # Tính Permutation Importance
    perm_importance = permutation_importance(model,X_train_scaled, y_train, n_repeats=8, random_state=42)
    
    # Tạo DataFrame cho dữ liệu quan trọng
    importance_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': perm_importance.importances_mean
    }).sort_values(by='Importance', ascending=False)
    
    # Dựa trên số lượng feature, điều chỉnh kích thước của biểu đồ
    num_features = len(importance_df)
    figsize = (10, num_features * 0.3)  # Điều chỉnh chiều cao dựa trên số feature
    
    # Vẽ biểu đồa
    importance_df.plot(kind='barh', x='Feature', y='Importance', legend=False, figsize=figsize)
    plt.title("Permutation Importance")
    plt.xlabel("Importance")
    plt.ylabel("Feature")
    plt.show()

    return importance_df

# Lasso

In [ ]:
import pandas as pd
from sklearn.linear_model import LassoCV
import numpy as np
import matplotlib.pyplot as plt

def lasso_feature_selection(X_train, X_train_scaled, y_train):
    # Lasso Regression với Cross-Validation để tìm alpha tốt nhất
    lasso = LassoCV(cv=5, random_state=42)
    lasso.fit(X_train_scaled, y_train)
    
    # In ra trọng số của các feature
    feature_importance = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': np.abs(lasso.coef_)
    }).sort_values(by='Importance', ascending=False)
    
    # Tính chiều cao dựa trên số lượng feature
    num_features = len(feature_importance)
    height = max(4, num_features * 0.3)  # đảm bảo không quá nhỏ
    
    # Vẽ biểu đồ trọng số của feature
    feature_importance.plot(kind='barh', x='Feature', y='Importance', legend=False, figsize=(8, height))
    plt.title("Lasso Feature Importance")
    plt.xlabel("Importance")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.show()
    
    return feature_importance

# Boruta Trick

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def boruta_trick(model, X_train, X_train_scaled, y_train):
    # Tạo một feature ngẫu nhiên (feature "bóng") và scale nó tương tự
    random_feature = np.random.rand(X_train.shape[0], 1)

    # Tính min và max của X_train_scaled (giả sử scale về [0, 1] hoặc tương tự)
    min_val = np.min(X_train_scaled)
    max_val = np.max(X_train_scaled)

    # Scale random feature về cùng khoảng
    scaled_random_feature = min_val + random_feature * (max_val - min_val)

    X_train_random_scaled = np.hstack([X_train_scaled, scaled_random_feature])
    feature_names = list(X_train.columns) + ['Random Feature']

    # Huấn luyện mô hình trên dữ liệu đã scale có thêm feature ngẫu nhiên
    model.fit(X_train_random_scaled, y_train)

    # Tính toán importance của từng feature
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False).reset_index(drop=True)

    # In ra độ quan trọng của tất cả các feature (bao gồm feature ngẫu nhiên)
    print('Feature Importance (Including Random Feature):')
    print(feature_importance)

    # Xác định ngưỡng quan trọng là độ quan trọng của feature ngẫu nhiên
    random_feature_importance = feature_importance[feature_importance['Feature'] == 'Random Feature']['Importance'].iloc[0]

    # Lọc các feature quan trọng hơn feature ngẫu nhiên
    important_features = feature_importance[feature_importance['Importance'] > random_feature_importance]
    print('\nImportant features (Above Random Feature):')
    print(important_features)

    # Dựa trên số lượng feature, điều chỉnh kích thước của biểu đồ
    num_features = len(feature_importance)
    figsize = (10, max(6, num_features * 0.4))  # Điều chỉnh chiều cao dựa trên số feature
                                                 # Đảm bảo chiều cao tối thiểu là 6

    # Vẽ biểu đồ
    plt.figure(figsize=figsize)
    plt.barh(feature_importance['Feature'], feature_importance['Importance'])
    plt.title("Boruta Feature Importance (Trick) - Scaled Data")
    plt.xlabel("Importance")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.show()

    return important_features


# Calibration

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

def calibrate_model(model, X_train, y_train, method='sigmoid'):
    """
    Áp dụng Calibration (One-vs-Rest cho đa lớp).
    
    Parameters:
    model: mô hình đã huấn luyện
    X_train: Dữ liệu huấn luyện
    y_train: Nhãn huấn luyện
    method: phương pháp calibration ('sigmoid' hoặc 'isotonic')
    
    Return:
    calibrated_model: mô hình đã qua calibration
    """
    calibrated_model = CalibratedClassifierCV(model, method=method, cv='prefit')
    calibrated_model.fit(X_train, y_train)
    return calibrated_model

In [ ]:
# SHAP (SHapley Additive exPlanations)
import shap

def plot_shap_importance(model, X_train, X_train_scaled):
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_train_scaled)
    # Vẽ SHAP summary plot
    shap.summary_plot(shap_values, X_train)
    return shap_values

In [ ]:
import shap
import pandas as pd
import numpy as np

def plot_shap_importance_svm(model, X_train, X_train_scaled):
    """
    Tính toán và vẽ SHAP importance cho mô hình SVM bằng KernelExplainer.

    Args:
        model: Mô hình SVM đã được huấn luyện.
        X_train (pd.DataFrame): DataFrame chứa các đặc trưng huấn luyện (chưa scale).
        X_train_scaled (np.ndarray): Mảng NumPy chứa các đặc trưng huấn luyện đã scale.

    Returns:
        np.ndarray: Mảng chứa các giá trị SHAP.
    """
    # Sử dụng KernelExplainer cho các mô hình không dựa trên cây
    explainer = shap.KernelExplainer(model.predict_proba, X_train_scaled)
    shap_values = explainer.shap_values(X_train_scaled)

    # Vẽ SHAP summary plot
    # Lưu ý: KernelExplainer trả về một list các mảng SHAP values cho từng lớp
    # Chúng ta có thể chọn một lớp cụ thể để hiển thị hoặc lấy trung bình
    if model.probability:
        shap.summary_plot(shap_values, X_train)
    else:
        print("Mô hình SVM cần được huấn luyện với probability=True để sử dụng predict_proba cho SHAP.")

    return shap_values

In [ ]:
def train_evaluate_svm_per_phase_v2(phases):
    """
    Huấn luyện và đánh giá mô hình SVM cho từng phase dữ liệu.
    Phiên bản này loại bỏ các cột 'total_score', 'label', 'label_encoded'
    khỏi features trước khi xử lý.

    Args:
        phases (list): Một danh sách các tuple, mỗi tuple chứa (train_df, test_df)
                       cho một phase cụ thể.
    """
    all_results = {}

    for i, (train_df, test_df) in enumerate(phases):
        phase_name = f"Phase {i+1}"
        print(f"\n--- Processing {phase_name} ---")

        # 2. Chuẩn bị dữ liệu
        # Tách features và target
        columns_to_drop = ['total_score', 'label', 'label_encoded']
        X_train = train_df.drop(columns=columns_to_drop, axis=1).copy()
        X_test = test_df.drop(columns=columns_to_drop, axis=1).copy()

        # Lấy nhãn (target) từ cột 'label_encoded'
        y_train = train_df.pop("label_encoded").copy()
        y_test = test_df.pop("label_encoded").copy()

        # Loại bỏ cột 'user_id'
        if 'user_id' in X_train.columns:
            X_train.drop('user_id', axis=1, inplace=True)
        if 'user_id' in X_test.columns:
            X_test.drop('user_id', axis=1, inplace=True)

        # Xử lý NaN trong cột 'school'
        if 'school' in X_train.columns:
            X_train['school'].fillna('Unknown', inplace=True)
        if 'school' in X_test.columns:
            X_test['school'].fillna('Unknown', inplace=True)

        # Frequency Encoding cho 'school'
        if 'school' in X_train.columns:
            school_freq = X_train['school'].value_counts(normalize=True)
            X_train['school_encoded'] = X_train['school'].map(school_freq)
            X_test['school_encoded'] = X_test['school'].map(school_freq).fillna(0)
            X_train.drop('school', axis=1, inplace=True)
            X_test.drop('school', axis=1, inplace=True)

        # Frequency Encoding cho 'course_id'
        if 'course_id' in X_train.columns:
            course_freq = X_train['course_id'].value_counts(normalize=True)
            X_train['course_id_encoded'] = X_train['course_id'].map(course_freq)
            X_test['course_id_encoded'] = X_test['course_id'].map(course_freq).fillna(0)
            X_train.drop('course_id', axis=1, inplace=True)
            X_test.drop('course_id', axis=1, inplace=True)

        # Xác định các cột số để chuẩn hóa
        numerical_cols_train = X_train.select_dtypes(include=['number']).columns

        # Chuẩn hóa features số
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train[numerical_cols_train])
        X_test_scaled = scaler.transform(X_test[numerical_cols_train])

        # Huấn luyện mô hình SVM
        model = SVC(kernel='rbf', C=10, gamma=0.01, class_weight='balanced', random_state=42, probability=True)
        model.fit(X_train_scaled, y_train)

        # Dự đoán trên tập kiểm tra
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)

        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        print(f'Accuracy: {accuracy:.4f}')
        print(f'Precision: {precision:.4f}')
        print(f'Recall: {recall:.4f}')
        print(f'F1-score: {f1:.4f}')
        print('\nClassification Report:')
        print(classification_report(y_test, y_pred, zero_division=0))

        # Vẽ confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=model.classes_, yticklabels=model.classes_)
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.title(f'Confusion Matrix - {phase_name}')
        plt.show()

        # Tính ROC AUC (nếu phù hợp cho đa lớp)
        try:
            roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='weighted')
            print(f'ROC AUC: {roc_auc:.4f}')
            all_results[phase_name] = {
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1-score': f1,
                'roc_auc': roc_auc,
                'confusion_matrix': cm,
                'classification_report': classification_report(y_test, y_pred, zero_division=0, output_dict=True)
            }
        except ValueError as e:
            print(f"Không thể tính ROC AUC: {e}")
            all_results[phase_name] = {
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1-score': f1,
                'confusion_matrix': cm,
                'classification_report': classification_report(y_test, y_pred, zero_division=0, output_dict=True)
            }

        # Lưu kết quả
        results_df = X_test.copy()
        results_df['True_Label'] = y_test
        results_df['Predicted_Label'] = y_pred
        results_filename = f"test_results_standard_accuracy_{phase_name}.csv"
        results_df.to_csv(results_filename, index=False)
        print(f"Results saved as {results_filename}")

    
        # Calibrate model
        calibrate_train_model = calibrate_model(model, X_train_scaled, y_train)
        y_calibrate_pred = calibrate_train_model.predict(X_test_scaled)
    
        results_df = X_test.copy()
        results_df['True_Label'] = y_test
        results_df['Predicted_Label'] = y_calibrate_pred
        results_filename = f"test_results_calibrate_standard_accuracy_{phase_name}.csv"
        results_df.to_csv(results_filename, index=False)
        print(f"Results saved as {results_filename}")

    
        # Feature selection và SHAP (chỉ với phase 1-2)
        if i <= 2:
            print(f"\n-------------Permutation Importance for phase: {phase_name}---------")
            perm_importance = plot_permutation_importance(model, X_train, X_train_scaled, y_train)
            print(perm_importance)
            
            # print(f"\n-----------SHAP Feature Importance for phase: {phase_name}------------")
            # shap_values = plot_shap_importance_svm(model, X_train, X_train_scaled)
    
            print(f"\n-----Lasso Feature Selection for phase: {phase_name}----------")
            lasso_importance = lasso_feature_selection(X_train, X_train_scaled, y_train)
            print(lasso_importance)
    
            # print(f"\n----------Boruta Trick: {phase_name}----------")
            # important_features = boruta_trick(model,X_train,  X_train_scaled, y_train)
            # print(important_features)
            # if i==1:
            #     return all_results

    return all_results

In [ ]:
if __name__ == '__main__':
    # Đọc dữ liệu từ các file CSV
    final_phase1_train_df = pd.read_csv("/kaggle/input/final-data/phase1/user_train_phase_1_train.csv")
    final_phase2_train_df = pd.read_csv("/kaggle/input/final-data/phase2/user_train_phase_2_train.csv")
    final_phase3_train_df = pd.read_csv("/kaggle/input/final-data/phase3/user_train_phase_3_train.csv")
    final_phase4_train_df = pd.read_csv("/kaggle/input/final-data/phase4/user_train_phase_4_train.csv")

    final_phase1_test_df = pd.read_csv("/kaggle/input/final-data/phase1/user_train_phase_1_test.csv")
    final_phase2_test_df = pd.read_csv("/kaggle/input/final-data/phase2/user_train_phase_2_test.csv")
    final_phase3_test_df = pd.read_csv("/kaggle/input/final-data/phase3/user_train_phase_3_test.csv")
    final_phase4_test_df = pd.read_csv("/kaggle/input/final-data/phase4/user_train_phase_4_test.csv")

    phases_data = [
        (final_phase1_train_df, final_phase1_test_df),
        (final_phase2_train_df, final_phase2_test_df),
        (final_phase3_train_df, final_phase3_test_df),
        (final_phase4_train_df, final_phase4_test_df)
    ]

    results = train_evaluate_svm_per_phase_v2(phases_data)

    # In kết quả tổng hợp (tùy chọn)
    print("\n--- Summary of Results ---")
    for phase, metrics in results.items():
        print(f"\n{phase}:")
        print(f"  Accuracy: {metrics['accuracy']:.4f}")
        print(f"  Precision: {metrics['precision']:.4f}")
        print(f"  Recall: {metrics['recall']:.4f}")
        print(f"  F1-score: {metrics['f1-score']:.4f}")
        if 'roc_auc' in metrics:
            print(f"  ROC AUC: {metrics['roc_auc']:.4f}")
        print(f"  Confusion Matrix:\n{metrics['confusion_matrix']}")
        print(f"  Classification Report:\n{metrics['classification_report']}")